In [70]:
import openai
import dotenv
import os

if not dotenv.load_dotenv("../../.env"):
    raise ValueError("EnvironmentError: Failed to load `.env`")

api_key = os.getenv("OPENAI_API_KEY") or ""

if not api_key:
    raise ValueError("EnvironmentError: Failed to load `OPENAI_API_KEY`")

openai.api_key = api_key

In [71]:
system_prompt = {
    "role": "system", "content": "My name is ChatGPT. I am a helpful assistant."
}

user_input = {
    "role": "user", "content": "Hello! What is your name?"
}

messages = [system_prompt, user_input]

get_chat_completion = openai.ChatCompletion.create

response = get_chat_completion(model="gpt-3.5-turbo", messages=messages, temperature=0)

messages.append(response["choices"][0]["message"])

print(response)

{
  "id": "chatcmpl-7ec9TyoMRjNU6HUXMPxk7NGWL8Lek",
  "object": "chat.completion",
  "created": 1689914131,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! My name is ChatGPT. How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 31,
    "completion_tokens": 16,
    "total_tokens": 47
  }
}


In [72]:
for message in messages:
    print(f"{message['role']}\n{message['content']}\n")

system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?



In [73]:
functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": { "type": "string", "enum": ["celsius", "fahrenheit"] }
        },
        "required": ["location"]
      }
    }
]


def get_current_weather(location: str, unit: str = "celsius"):
    """
    Get the current weather in a given location.

    Parameters:
    location (str): The city and state, e.g. San Francisco, CA
    unit (str): The unit of temperature, can be either 'celsius' or 'fahrenheit'. Default is 'celsius'.

    Returns:
    str: A string that describes the current weather.
    """

    # This is a mock function, so let's return a mock weather report.
    weather_report = f"The current weather in {location} is 20 degrees {unit}."
    return weather_report


In [74]:
user_input = {
    "role": "user", "content": "What is the weather like today in New York City, New York?"
}

messages.append(user_input)

response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

messages.append(response["choices"][0]["message"])

print(response)

{
  "id": "chatcmpl-7ecACKpMd4fq35Am1fWZuD66w42a9",
  "object": "chat.completion",
  "created": 1689914176,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"New York, NY\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 125,
    "completion_tokens": 19,
    "total_tokens": 144
  }
}


In [76]:
for message in messages:
    print(f"{message['role']}\n{message['content']}\n")

system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?

user
What is the weather like today?

assistant
None



In [77]:
function_call = response["choices"][0]["message"]["function_call"]

print(function_call)

{
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"New York, NY\"\n}"
}


In [78]:
import json

function_name = function_call["name"]
function_args = json.loads(function_call["arguments"])

print(function_name, function_args)

get_current_weather {'location': 'New York, NY'}


In [79]:
if function_name == 'get_current_weather':
    location = function_args['location']
    unit = function_args.get('unit', 'celsius')  # use default 'celsius' if unit is not provided
    weather_report = get_current_weather(location, unit)
    # then pass weather_report back to the model as a new message
    print(weather_report)

The current weather in New York, NY is 20 degrees celsius.


In [81]:
messages[-1]["content"] = weather_report

for message in messages:
    print(f"{message['role']}\n{message['content']}\n")

system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?

user
What is the weather like today?

assistant
The current weather in New York, NY is 20 degrees celsius.



In [82]:
messages.append({"role": "user", "content": "Thank you! How should I dress for that kind of temperature?"})

for message in messages:
    print(f"{message['role']}\n{message['content']}\n")


system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?

user
What is the weather like today?

assistant
The current weather in New York, NY is 20 degrees celsius.

user
Thank you! How should I dress for that kind of temperature?



In [83]:
# Call the model again with the updated messages
response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

print(response)

{
  "id": "chatcmpl-7ecEBbVOOPN0fDEPUqQVGttw50TqM",
  "object": "chat.completion",
  "created": 1689914423,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "For a temperature of 20 degrees Celsius, you might consider wearing a light jacket or sweater. You could also layer your clothing to adjust for any changes in temperature throughout the day. Don't forget to check the weather forecast for any possible rain or wind, as this may impact your clothing choice as well."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 183,
    "completion_tokens": 62,
    "total_tokens": 245
  }
}


In [84]:
messages.append(response["choices"][0]["message"])

for message in messages:
    print(f"{message['role']}\n{message['content']}\n")

system
My name is ChatGPT. I am a helpful assistant.

user
Hello! What is your name?

assistant
Hello! My name is ChatGPT. How can I assist you today?

user
What is the weather like today?

assistant
The current weather in New York, NY is 20 degrees celsius.

user
Thank you! How should I dress for that kind of temperature?

assistant
For a temperature of 20 degrees Celsius, you might consider wearing a light jacket or sweater. You could also layer your clothing to adjust for any changes in temperature throughout the day. Don't forget to check the weather forecast for any possible rain or wind, as this may impact your clothing choice as well.

